In [79]:
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [80]:
data = pd.read_csv('data/Zoo.csv')
# feature_columns = ['target','id','hair','feathers','eggs','milk',
#                    'airborne','aquatic','predator','toothed','backbone','breathes',
#                    'venomous','fins','legs','tail','domestic','catsize']

In [81]:
data

id  target  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0      1       1     1         0     0     1         0        0         1   
1      2       1     1         0     0     1         0        0         0   
2      3       4     0         0     1     0         0        1         1   
3      4       1     1         0     0     1         0        0         1   
4      5       1     1         0     0     1         0        0         1   
..   ...     ...   ...       ...   ...   ...       ...      ...       ...   
96    96       1     1         0     0     1         0        0         0   
97    97       6     1         0     1     0         1        0         0   
98    98       1     1         0     0     1         0        0         1   
99    99       7     0         0     1     0         0        0         0   
100  100       2     0         1     1     0         1        0         0   

     toothed  backbone  breathes  venomous  fins  legs  tail  domestic  \
0          1         1         1         0     0     4     0         0   
1          1         1         1         0     0     4     1         0   
2          1         1         0         0     1     0     1         0   
3          1         1         1         0     0     4     0         0   
4          1         1         1         0     0     4     1         0   
..       ...       ...       ...       ...   ...   ...   ...       ...   
96         1         1         1         0     0     2     1         0   
97         0         0         1         1     0     6     0         0   
98         1         1         1         0     0     4     1         0   
99         0         0         1         0     0     0     0         0   
100        0         1         1         0     0     2     1         0   

     catsize  
0          1  
1          1  
2          0  
3          1  
4          1  
..       ...  
96         1  
97         0  
98         1  
99         0  
100        0  

[101 rows x 18 columns]

In [82]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [83]:
import math
def fitness_function(condition_attrs, data, decision_attr):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data, condition_attrs)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if(len(x)>total_records*0.01):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return 999;
    if total_rule/total_records<0.4:
        return 999

    avg_confidence = np.sum(rule_confidences) / count +0.1*(entropy_D-entropy_DB)

    return avg_confidence

In [84]:
def generate_population(size, feature_columns):
    # 初始化种群
    population = []
    for i in range(size):
        individual = random.sample(feature_columns, random.randint(3, len(feature_columns) - 1))
        population.append(individual)
    return population

def select_parents(population, fitness_values, elite_size):
    elite_indices = np.argsort(fitness_values)[:elite_size]
    elite_population = [population[i] for i in elite_indices]
    population_without_elite = [individual for i, individual in enumerate(population) if i not in elite_indices]
    # 将相应的适应度函数值也从 fitness_values 中删除
    fitness_values_without_elite = [fitness_values[i] for i in range(len(fitness_values)) if i not in elite_indices]
    probabilities = (1 / (np.array(fitness_values_without_elite) + [1e-10])) / sum(1 / (np.array(fitness_values_without_elite) + [1e-10]))
    parents = []
    for _ in range(len(population) - elite_size):
        parent1_index = np.random.choice(len(population_without_elite), p=probabilities)
        parent2_index = np.random.choice(len(population_without_elite), p=probabilities)
        parents.append(population_without_elite[parent1_index])
        parents.append(population_without_elite[parent2_index])
    parents.extend(elite_population)
    return parents


def crossover(parents, crossover_prob):
    offspring = []
    if(len(parents)%2!=0):
        parents.pop()
    for i in range(0, len(parents)-1, 2):
        if random.random() < crossover_prob:
            # 随机选择交叉点，注意不能选择第一个或最后一个位置
            crossover_point = random.randint(1, len(parents[i]) - 2)
            # 生成两个新的后代
            child1 = parents[i][:crossover_point] + parents[i+1][crossover_point:]
            child2 = parents[i+1][:crossover_point] + parents[i][crossover_point:]
            offspring.extend([child1, child2])
        else:
            # 如果没有进行交叉操作，则直接将两个父代加入后代中
            offspring.extend([parents[i], parents[i+1]])
    return offspring

def mutate(individual, mutation_prob, feature_columns):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual) - 1):
        if random.random() < mutation_prob:
            attr = mutated_individual[i]
            other_attrs = [col for col in feature_columns if col != attr]
            if other_attrs:
                mutated_individual[i] = random.choice(other_attrs)
    return mutated_individual

In [85]:
def run_genetic_algorithm(data, decision_attr, population_size=50, max_generations=100, crossover_prob=0.8, mutation_prob=0.2, elite_size=5):
    feature_columns = [col for col in data.columns if col not in ['target','id']]
    # 初始化种群
    population = generate_population(population_size,feature_columns)
    # 计算每个个体的适应度函数值

    fitness_values = []
    for feature_columns in tqdm(population):
        fitness = fitness_function(feature_columns, data, decision_attr)
        fitness_values.append(fitness)
    # fitness_values = [fitness_function(feature_columns, data, decision_attr) for feature_columns in population]
    # 记录最优个体的适应度函数值和基因型
    best_fitness_value = min(fitness_values)
    best_individual = population[np.argmin(fitness_values)]
    # 进化过程
    for generation in tqdm(range(max_generations)):
        # 选择父母
        selected_parents = select_parents(population, fitness_values,elite_size)
        # 按一定概率进行交叉操作
        offspring = crossover(selected_parents, crossover_prob)
        # 按一定概率进行变异操作
        offspring = [mutate(individual, mutation_prob, feature_columns) for individual in offspring]
        # 计算每个个体的适应度函数值
        offspring_fitness_values = [fitness_function(individual, data, decision_attr) for individual in offspring]

        # 合并父代和子代
        combined_population = population + offspring
        combined_fitness_values = fitness_values + offspring_fitness_values

        # 按适应度函数值从小到大排序
        sorted_indices = np.argsort(combined_fitness_values)

        combined_population = [combined_population[i] for i in sorted_indices]
        combined_fitness_values = [combined_fitness_values[i] for i in sorted_indices]

        # 更新最优个体
        if combined_fitness_values[0] < best_fitness_value:
            best_fitness_value = combined_fitness_values[0]
            best_individual = combined_population[0]
        print("第轮最优个体")
        print(best_individual)
        print(best_fitness_value)
        # 选择新的种群
        population = combined_population[:population_size]
        fitness_values = [fitness_function(individual, data, decision_attr) for individual in population]
    return best_individual, best_fitness_value

In [86]:
import time
start_time = time.time()
print(run_genetic_algorithm(data,'target', population_size=30, max_generations=60, crossover_prob=0.8,
                            mutation_prob=0.01, elite_size=1))
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

  0%|          | 0/60 [00:00<?, ?it/s]

第轮最优个体
['airborne', 'venomous', 'breathes', 'predator', 'legs', 'toothed', 'eggs', 'milk', 'feathers', 'hair', 'tail', 'aquatic', 'fins', 'domestic']
0.23905596822940392


  2%|▏         | 1/60 [00:01<01:13,  1.25s/it]

第轮最优个体
['airborne', 'venomous', 'breathes', 'predator', 'legs', 'toothed', 'eggs', 'milk', 'feathers', 'hair', 'tail', 'aquatic', 'fins', 'domestic']
0.23905596822940392


  3%|▎         | 2/60 [00:02<01:16,  1.32s/it]

...

第轮最优个体
['airborne', 'venomous', 'breathes', 'predator', 'legs', 'toothed', 'eggs', 'milk', 'feathers', 'hair', 'tail', 'aquatic', 'fins', 'domestic']
0.23905596822940392


 23%|██▎       | 14/60 [00:19<01:04,  1.39s/it]

第轮最优个体
['airborne', 'venomous', 'breathes', 'predator', 'legs', 'toothed', 'eggs', 'milk', 'feathers', 'hair', 'tail', 'aquatic', 'fins', 'domestic']
0.23905596822940392


 25%|██▌       | 15/60 [00:21<01:04,  1.43s/it]


KeyboardInterrupt: 

In [87]:
feature_columns=['airborne', 'venomous', 'breathes', 'predator', 'legs', 'toothed', 'eggs', 'milk', 'feathers', 'hair', 'tail', 'aquatic', 'fins', 'domestic']
len(feature_columns)

14